# TP1 - Parte 1 - Procesos

# **Python**

Arbol de procesos en Python

In [ ]:
%%writefile fork.py
import os
import sys
import time

def parameter_validation():
    if len(sys.argv) == 2 and (sys.argv[1] == "-help" or sys.argv[1] == "-h" or sys.argv[1] == "-?"):
        print("Para correr el script: python ./fork.py\n")
        print("Para verificar el correcto funcionamiento y ver el arbol de procesos utilizar: pstree nombre-del-proceso\n")
        sys.exit(1)
    elif len(sys.argv) >= 2:
        print("Error, cantidad de parametros inadecuados, para ayuda: python ./fork.py -h\n")
        sys.exit(1)

def show_process(pid, ppid, id_process):
    print("Id:", id_process, "PID:", pid, "PPID:", ppid)

def main_process():
    pidh1 = os.fork()
    if pidh1 == 0:
        pidn1 = os.fork()
        if pidn1 == 0:
            show_process(os.getpid(), os.getppid(), 'E')
        else:
            pidn2 = os.fork()
            if pidn2 == 0:
                pidb1 = os.fork()
                if pidb1 == 0:
                    show_process(os.getpid(), os.getppid(), 'H')
                else:
                    pidb2 = os.fork()
                    if pidb2 == 0:
                        pidt1 = os.fork()
                        if pidt1 == 0:
                            show_process(os.getpid(), os.getppid(), 'J')
                        else:
                            show_process(os.getpid(), os.getppid(), 'I')
                    else:
                        show_process(os.getpid(), os.getppid(), 'F')
            else:
                show_process(os.getpid(), os.getppid(), 'B')
    else:
        pidh2 = os.fork()
        if pidh2 == 0:
            show_process(os.getpid(), os.getppid(), 'C')
        else:
            pidh3 = os.fork()
            if pidh3 == 0:
                pidn3 = os.fork()
                if pidn3 == 0:
                    show_process(os.getpid(), os.getppid(), 'G')
                else:
                    show_process(os.getpid(), os.getppid(), 'D')
            else:
                show_process(os.getpid(), os.getppid(), 'A')
    time.sleep(30)

if __name__ == "__main__":
    parameter_validation()
    main_process()


Writing fork.py


In [ ]:
!chmod 700 ./fork.py

In [ ]:
!nohup python -u fork.py 1>salidaPython 2>/dev/null &

In [ ]:
!cat salidaPython

Id: A PID: 37453 PPID: 1
Id: B PID: 37455 PPID: 37453
Id: C PID: 37456 PPID: 37453
Id: E PID: 37457 PPID: 37455
Id: F PID: 37459 PPID: 37455
Id: I PID: 37461 PPID: 37459
Id: D PID: 37458 PPID: 37453
Id: J PID: 37462 PPID: 37461
Id: H PID: 37460 PPID: 37459
Id: G PID: 37463 PPID: 37458


In [ ]:
!ps -ef | grep fork.py


root       37453       1  0 02:48 ?        00:00:00 python3 -u fork.py
root       37455   37453  0 02:48 ?        00:00:00 python3 -u fork.py
root       37456   37453  0 02:48 ?        00:00:00 python3 -u fork.py
root       37457   37455  0 02:48 ?        00:00:00 python3 -u fork.py
root       37458   37453  0 02:48 ?        00:00:00 python3 -u fork.py
root       37459   37455  0 02:48 ?        00:00:00 python3 -u fork.py
root       37460   37459  0 02:48 ?        00:00:00 python3 -u fork.py
root       37461   37459  0 02:48 ?        00:00:00 python3 -u fork.py
root       37462   37461  0 02:48 ?        00:00:00 python3 -u fork.py
root       37463   37458  0 02:48 ?        00:00:00 python3 -u fork.py
root       37502     230  0 02:48 ?        00:00:00 /bin/bash -c ps -ef | grep fork.py
root       37504   37502  0 02:48 ?        00:00:00 grep fork.py


In [ ]:
!pstree -pc $(pgrep -f fork.py -P 1)

python3─┬─python3─┬─python3
        │         └─python3─┬─python3
        │                   └─python3───python3
        ├─python3
        └─python3───python3


# **C++**
Arbol de procesos en C++

In [ ]:
%%writefile fork.cpp
#include <iostream>
#include <map>
#include <stack>
#include <unistd.h>
#include <sys/wait.h>

struct TreeMap;
using BranchMap = std::map<char, TreeMap>;
struct TreeMap
{
    BranchMap branchMap;
};

void iterate(BranchMap parentBranchMap, pid_t parentPID, char parent)
{
    for (const auto &branchMapParentIterator : parentBranchMap)
    {
        pid_t pid = fork();
        char me = branchMapParentIterator.first;
        if (pid == 0)
        {
            iterate(branchMapParentIterator.second.branchMap, getpid(), me);

        }
        else
        {
            std::cout << "Parent: " << parent << " Me: " << me << " ParentPID: " << parentPID << " mePID: " << pid << std::endl;

        }
    }
    sleep(30);
    exit(0);
}

int main()
{
    TreeMap treeMap;
    treeMap.branchMap['b'].branchMap['e'];
    treeMap.branchMap['c'];
    treeMap.branchMap['d'].branchMap['g'];
    treeMap.branchMap['b'].branchMap['f'].branchMap['h'];
    treeMap.branchMap['b'].branchMap['f'].branchMap['i'].branchMap['j'];

    iterate(treeMap.branchMap, getpid(), 'a');
    std::cin.get(); // Wait for the user to press Enter
    return 0;
}

Writing fork.cpp


In [ ]:
!g++ fork.cpp -o fork

In [ ]:
!nohup ./fork 1>salidaCpp 2>/dev/null &

In [ ]:
!cat salidaCpp

Parent: a Me: b ParentPID: 1014 mePID: 1015
Parent: a Me: c ParentPID: 1014 mePID: 1016
Parent: a Me: d ParentPID: 1014 mePID: 1017
Parent: b Me: e ParentPID: 1015 mePID: 1018
Parent: b Me: f ParentPID: 1015 mePID: 1019
Parent: d Me: g ParentPID: 1017 mePID: 1020
Parent: f Me: h ParentPID: 1019 mePID: 1021
Parent: f Me: i ParentPID: 1019 mePID: 1022
Parent: i Me: j ParentPID: 1022 mePID: 1023


In [ ]:
!ps -ef | grep fork

root        1014       1  0 10:44 ?        00:00:00 ./fork
root        1015    1014  0 10:44 ?        00:00:00 ./fork
root        1016    1014  0 10:44 ?        00:00:00 ./fork
root        1017    1014  0 10:44 ?        00:00:00 ./fork
root        1018    1015  0 10:44 ?        00:00:00 ./fork
root        1019    1015  0 10:44 ?        00:00:00 ./fork
root        1020    1017  0 10:44 ?        00:00:00 ./fork
root        1021    1019  0 10:44 ?        00:00:00 ./fork
root        1022    1019  0 10:44 ?        00:00:00 ./fork
root        1023    1022  0 10:44 ?        00:00:00 ./fork
root        1049     182  0 10:45 ?        00:00:00 /bin/bash -c ps -ef | grep fork
root        1051    1049  0 10:45 ?        00:00:00 grep fork


In [ ]:
!pstree -pc $(pgrep -f fork -P 1)

fork(1014)─┬─fork(1015)─┬─fork(1018)
           │            └─fork(1019)─┬─fork(1021)
           │                         └─fork(1022)───fork(1023)
           ├─fork(1016)
           └─fork(1017)───fork(1020)


# **Java**
Arbol de procesos en Java

In [1]:
%%writefile ParentProcess.java
import java.io.IOException;

public class ParentProcess
{
    /*Constantes*/
    private static final String JAVA_FILE_LOCATION="ChildProcess.java";
    private static final int numberOfChilds = 3;
    /*Recursos necesarios*/
    static Process childs[];
    static String[] letter = {"B","C","D"};


    public static void createChilds(int numberOfChilds) throws IOException
    {
        ProcessBuilder builder;
        childs = new Process[numberOfChilds];
        for(int i=0;i<numberOfChilds;i++)
        {
            builder = new ProcessBuilder("java",JAVA_FILE_LOCATION,letter[i]); //Acá ya le paso la letra del hijo
            builder.inheritIO();
            childs[i] = builder.start();
        }
    }

    public static void waitChilds(int numberOfChilds) throws InterruptedException
    {
        for(int i=0; i<numberOfChilds; i++)
        {
            childs[i].waitFor();
        }
    }

    public static void main(String[] args) throws IOException,InterruptedException
    {
        ProcessHandle yo = ProcessHandle.current();
        long pid = yo.pid();
        System.out.println("A PID: "+pid); //En este caso, siempre va a ser A, por ser el proceso Padre de todo
        createChilds(numberOfChilds);
        waitChilds(numberOfChilds);
    }
}

Writing ParentProcess.java


In [2]:
%%writefile ChildProcess.java
import java.io.IOException;
import java.util.ArrayList;

public class ChildProcess

{
    /* Constantes*/
    static final String processB = "B";
    static final String processD = "D";
    static final String processE = "E";
    static final String processF = "F";
    static final String processG = "G";
    static final String processH = "H";
    static final String processI = "I";
    static final String processJ = "J";
    static final int time = 50000;
    /*Ruta del siguiente nivel*/
    private static final String JAVA_FILE_LOCATION="ChildProcess.java";
    /*Recursos necesarios*/
    static ArrayList<Process> childs = new ArrayList<Process>();

    public static void showInfo(String letter)
    {
        ProcessHandle yo = ProcessHandle.current();
        long pid = yo.pid();
        long ppid = yo.parent().get().pid();
        System.out.println(letter+" PID: "+pid+" PPID: "+ ppid);
    }

    public static void createChild(String child) throws IOException
    {
        ProcessBuilder builder = new ProcessBuilder("java", JAVA_FILE_LOCATION, child);
        builder.inheritIO();
        childs.add(builder.start());
    }

    public static void waitChilds() throws InterruptedException
    {
        for (Process c : childs)
        {
			c.waitFor();
		}
    }

    public static void main (String[] args) throws IOException,InterruptedException
    {
        showInfo(args[0]);

        switch (args[0])
        {
            case processB:
                createChild(processE);
                createChild(processF);
                break;
            case processD:
            	createChild(processG);
            	break;
            case processF:
            	createChild(processH);
            	createChild(processI);
            	break;
            case processI:
            	createChild(processJ);
            	break;
            default:
            	Thread.sleep(time);
                break;
        }
        waitChilds();
    }
}



Writing ChildProcess.java


In [3]:
!nohup java ParentProcess.java 1>salidaJava 2>/dev/null &

In [7]:
!cat salidaJava

A PID: 411
C PID: 440 PPID: 411
B PID: 438 PPID: 411
D PID: 445 PPID: 411
E PID: 514 PPID: 438
F PID: 516 PPID: 438
G PID: 548 PPID: 445
I PID: 594 PPID: 516
H PID: 591 PPID: 516
J PID: 644 PPID: 594


In [8]:
!ps -ef | grep java

root         411       1 12 17:25 ?        00:00:02 java ParentProcess.java
root         438     411 11 17:25 ?        00:00:02 java ChildProcess.java B
root         440     411 10 17:25 ?        00:00:02 java ChildProcess.java C
root         445     411 11 17:25 ?        00:00:02 java ChildProcess.java D
root         514     438 14 17:25 ?        00:00:02 java ChildProcess.java E
root         516     438 13 17:25 ?        00:00:02 java ChildProcess.java F
root         548     445 14 17:25 ?        00:00:02 java ChildProcess.java G
root         591     516 18 17:25 ?        00:00:02 java ChildProcess.java H
root         594     516 19 17:25 ?        00:00:02 java ChildProcess.java I
root         644     594 24 17:25 ?        00:00:01 java ChildProcess.java J
root         694     268  0 17:25 ?        00:00:00 /bin/bash -c ps -ef | grep java
root         696     694  0 17:25 ?        00:00:00 grep java


In [9]:
!pstree -pT $(pgrep -f ParentProcess.java -P 1)

java(411)─┬─java(438)─┬─java(514)
          │           └─java(516)─┬─java(591)
          │                       └─java(594)───java(644)
          ├─java(440)
          └─java(445)───java(548)
